## Preprocessing

In [4]:
import pandas as pd, numpy as np, tensorflow as tf, os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
# Read CSV
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")

# 1.  Drop IDs
application_df = application_df.drop(columns=["EIN", "NAME"])

# 2.  Log-scale ASK_AMT (skew fix)
application_df["ASK_AMT"] = np.log1p(application_df["ASK_AMT"])

# 3.  MODERATE rare-bucket cut-offs (back to 500 / 1 000)
for col, cutoff in [("APPLICATION_TYPE", 500), ("CLASSIFICATION", 1000)]:
    rare = application_df[col].value_counts().loc[lambda s: s < cutoff].index
    application_df[col] = application_df[col].replace(rare, "Other")

# 4.  One-hot encode
dummies = pd.get_dummies(application_df)

y = dummies["IS_SUCCESSFUL"]
X = dummies.drop(columns=["IS_SUCCESSFUL"])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)


## Compile, Train and Evaluate the Model

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers

input_dim = X_train_scaled.shape[1]

nn = tf.keras.Sequential([
    layers.Dense(512, activation="relu",
                 kernel_initializer="he_normal",
                 kernel_regularizer=regularizers.l2(1e-4),
                 input_dim=input_dim),
    layers.BatchNormalization(),

    layers.Dense(256, activation="relu",
                 kernel_initializer="he_normal",
                 kernel_regularizer=regularizers.l2(1e-4)),
    layers.BatchNormalization(),

    layers.Dense(128, activation="relu",
                 kernel_initializer="he_normal",
                 kernel_regularizer=regularizers.l2(1e-4)),

    layers.Dense(1, activation="sigmoid")
])

nn.compile(optimizer=tf.keras.optimizers.Adam(5e-4),
           loss="binary_crossentropy",
           metrics=["accuracy"])

nn.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 512)            │        22,528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 189,953 (742.00 KB)

 Trainable params: 188,417 (736.00 KB)

 Non-trainable params: 1,536 (6.00 KB)

In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os, math

early = EarlyStopping(monitor="val_loss",
                      patience=40,            # ↖ give it time
                      restore_best_weights=True)

os.makedirs("checkpoints_opt", exist_ok=True)
ckpt = ModelCheckpoint("checkpoints_opt/ep_{epoch:03d}.weights.h5",
                       save_weights_only=True)

history = nn.fit(
    X_train_scaled, y_train,
    validation_data=(X_test_scaled, y_test),
    epochs=400,
    batch_size=64,            # 32 → 64 stabilises BN stats
    callbacks=[early, ckpt],
    verbose=2
)

Epoch 1/400
429/429 - 8s - 19ms/step - accuracy: 0.7128 - loss: 0.7661 - val_accuracy: 0.7171 - val_loss: 0.7383
Epoch 2/400
429/429 - 4s - 9ms/step - accuracy: 0.7237 - loss: 0.7241 - val_accuracy: 0.7169 - val_loss: 0.7168
Epoch 3/400
429/429 - 4s - 9ms/step - accuracy: 0.7284 - loss: 0.7049 - val_accuracy: 0.7163 - val_loss: 0.7052
Epoch 4/400
429/429 - 6s - 14ms/step - accuracy: 0.7275 - loss: 0.6932 - val_accuracy: 0.7261 - val_loss: 0.6974
Epoch 5/400
429/429 - 3s - 8ms/step - accuracy: 0.7305 - loss: 0.6815 - val_accuracy: 0.7223 - val_loss: 0.6821
Epoch 6/400
429/429 - 6s - 13ms/step - accuracy: 0.7316 - loss: 0.6700 - val_accuracy: 0.7192 - val_loss: 0.6712
Epoch 7/400
429/429 - 4s - 10ms/step - accuracy: 0.7318 - loss: 0.6606 - val_accuracy: 0.7172 - val_loss: 0.6640
Epoch 8/400
429/429 - 4s - 10ms/step - accuracy: 0.7345 - loss: 0.6502 - val_accuracy: 0.7200 - val_loss: 0.6524
Epoch 9/400
429/429 - 6s - 15ms/step - accuracy: 0.7331 - loss: 0.6411 - val_accuracy: 0.7264 - val

In [8]:
loss, acc = nn.evaluate(X_test_scaled, y_test, verbose=0)
print(f"Validation accuracy: {acc:.4%} | loss: {loss:.4f}")

nn.save("AlphabetSoupCharity_Optimization.h5")
print("Saved model → AlphabetSoupCharity_Optimization.h5")


Validation accuracy: 72.7843% | loss: 0.5601
Saved model → AlphabetSoupCharity_Optimization.h5
